## ModelsTo Onnx

Dataset: [heartbeat categorization](https://www.kaggle.com/datasets/shayanfazeli/heartbeat?sort=votes)

## TODO - use with tf_env

In [8]:
import timeit, time
import numpy as np

### Yolo To Onnx

In [9]:
#from ultralytics import YOLO

In [10]:
from configparser import ConfigParser
conf = ConfigParser()
conf.read("../../api_keys.conf")

['../../api_keys.conf']

## Testcase 1: Image to Embeddings Conversion

In [11]:
from PIL import Image
import requests

### VGG-16 Tensorflow

[Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556) </br>


In [5]:
import subprocess

In [6]:
! CUDNN_PATH=$(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)")) && export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/:$CUDNN_PATH/lib

In [1]:
import tensorflow as tf 
tf.config.list_physical_devices()

2023-12-30 15:54:01.178445: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-30 15:54:01.241450: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 15:54:01.625731: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 15:54:01.628753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-30 15:54:04.199699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
base_model = VGG16(include_top=False, input_shape=(244,244,3))

Add a Global Average Pooling to Reduce to a 1-D embedding

In [4]:
model = Sequential()
for layer in base_model.layers:
    model.add(layer)
model.add(GlobalAveragePooling2D())

In [5]:
model.save("models/vgg16_embed.tf")

INFO:tensorflow:Assets written to: models/vgg16_embed.tf/assets


INFO:tensorflow:Assets written to: models/vgg16_embed.tf/assets


In [12]:
sample_img = Image.open(
    requests.get(
        "http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg", 
        stream=True)\
    .raw)

In [13]:
sample_img.size

(1800, 2400)

In [14]:
wh_ratio = sample_img.size[0]/sample_img.size[1]

In [15]:
IMG_DIM = (256, int(256*(1/wh_ratio)))
print(IMG_DIM)

(256, 341)


In [16]:
resized_sample_img = sample_img.resize((IMG_DIM))

In [17]:
sample_img_arr = np.asarray(resized_sample_img)

In [18]:
np.expand_dims(sample_img_arr,axis=0).shape

(1, 341, 256, 3)

In [19]:
image_gen = ImageDataGenerator()

In [20]:
input_sample = image_gen.flow(np.expand_dims(sample_img_arr,axis=0))

In [21]:
#input_tensor = convert_to_tensor(input_image)

In [22]:
type(input_sample)

keras.src.preprocessing.image.NumpyArrayIterator

In [23]:
start = time.monotonic()
sample_img_emb = model.predict(np.expand_dims(sample_img_arr,axis=0))
end = time.monotonic() - start 


ValueError: in user code:

    File "/home/james/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/james/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/james/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/james/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/james/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/james/anaconda3/envs/tf_env/lib/pytho

In [71]:
ms = round(end*1000)
print(ms)

152


In [72]:
from tensorflow.config import list_physical_devices 
list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

### Convert to Onnx Runtime

In [73]:
! pip install tf2onnx


In [74]:
import tf2onnx

In [75]:
import onnx

In [76]:
onnx_model = tf2onnx.convert.from_keras(model)

2023-09-17 14:44:58.137775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-17 14:44:58.147560: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-09-17 14:44:58.148564: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-09-17 14:44:58.149013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-17 14:44:58.149068: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

In [78]:
onnx.save(onnx_model[0], "./models/vgg16_cpu.onnx")

In [79]:
output_names = [n.name for n in onnx_model[0].graph.output]

In [80]:
output_names

['global_average_pooling2d_1']

In [81]:
import onnxruntime as ort 

In [82]:
sess = ort.InferenceSession("models/vgg16_cpu.onnx") #, #providers=["CUDAExecutionProvider"] if want to use GPU

In [84]:
start = time.monotonic()
onnx_pred = sess.run(output_names, {"input_3":np.expand_dims(sample_img_arr,axis=0)})
latency = time.monotonic() - start

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (tensor(uint8)) , expected: (tensor(float))

In [ ]:
transforms.Resize(IMG_SIZE),